In [71]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [72]:
def baixar_pagina(url):
  response = requests.get(url)
  response.encoding = "utf-8"
  html = response.text
  return html

## Globo Esporte

In [73]:
def extrair_conteudo_noticia(url):
    html = baixar_pagina(url)
    soup = BeautifulSoup(html, 'html.parser')
    ps = soup.find_all('p', class_='content-text__container')

    textos = []
    for p in ps:
        if p.find('div', class_='content-intertitle') or p.find('ul'):
            continue
        textos.append(p.get_text(" ", strip=True))
    return textos

In [74]:
from bs4 import BeautifulSoup
import pandas as pd

def extrair_noticias_globo_completo(url, time):

    html = baixar_pagina(url)
    soup = BeautifulSoup(html, 'html.parser')
    titulos_h2 = soup.find_all('h2')
    noticias = []

    for h2 in titulos_h2:
        a = h2.find('a')
        if not a:
            continue

        href = a.get('href')
        if not href:
            continue

        if href.startswith('/'):
            link = 'https://ge.globo.com' + href
        elif href.startswith('http://') or href.startswith('https://'):
            link = href
        else:
            continue

        titulo = a.get_text(strip=True)
        if not titulo:
            continue

        partes = extrair_conteudo_noticia(link)
        conteudo = " ".join(partes) if partes else None

        noticias.append({
            'time':     time,
            'titulo':   titulo,
            'link':     link,
            'conteudo': conteudo
        })

    return noticias


times = [
    "atletico-mg", "bahia", "botafogo", "bragantino", "ceara",
    "corinthians", "cruzeiro", "flamengo", "fluminense", "fortaleza",
    "gremio", "internacional", "juventude", "mirassol", "palmeiras",
    "santos", "sao-paulo", "sport", "vasco", "vitoria"
]

base_url = "https://ge.globo.com/futebol/times/"

noticias_globo = []
for t in times:
    url = base_url + t
    noticias_globo.extend(extrair_noticias_globo_completo(url, t))

df_globo = pd.DataFrame(noticias_globo)


## CNN

In [75]:
times = [
    "atletico-mineiro",
    "bahia",
    "botafogo",
    "ceara",
    "corinthians",
    "cruzeiro",
    "flamengo",
    "fluminense",
    "fortaleza",
    "gremio",
    "internacional",
    "juventude",
    "mirassol",
    "palmeiras",
    "red-bull-bragantino",
    "santos",
    "sao-paulo",
    "sport",
    "vasco",
    "vitoria",
]

In [76]:
from bs4 import BeautifulSoup

def extrair_noticias_cnn(url, time):
    html = baixar_pagina(url)
    soup = BeautifulSoup(html, 'html.parser')
    news = soup.find('ul', class_='home__new')
    noticias = []

    for item in news.find_all('li'):
        a  = item.find('a')
        h3 = item.find('h3')
        if a and h3:
            link   = a.get('href')
            titulo = h3.get_text(strip=True)
            if link and titulo:
                noticias.append({
                    'time':  time, 
                    'titulo': titulo,
                    'link':   link
                })

    return noticias


In [77]:
noticias_cnn = []

for i in range(len(times)):
    url = "https://www.cnnbrasil.com.br/esportes/futebol/" + times[i] + "/ultimas-noticias/"
    noticias = extrair_noticias_cnn(url, times[i])
    noticias_cnn.extend(noticias)

df_noticias_cnn = pd.DataFrame(noticias_cnn)

In [79]:
df = pd.concat([df_globo, df_noticias_cnn], ignore_index=True)
df = df[df['link'].str.contains('https://www.cnnbrasil.com.br/esportes/futebol/|https://ge.globo.com/futebol/times/')]
df


,time,titulo,link,conteudo
1,atletico-mg,"Assista tudo sobre o Galo na Globo, sportv e g...",https://ge.globo.com/futebol/times/atletico-mg...,None
2,atletico-mg,Júnior Santos supera duas lesões no Atlético-M...,https://ge.globo.com/futebol/times/atletico-mg...,"Nove jogos, duas lesões e quase quatro meses s..."
3,atletico-mg,Dudu comenta grama sintética do Atlético-MG e ...,https://ge.globo.com/futebol/times/atletico-mg...,Dudu foi apresentado oficialmente como jogador...
4,atletico-mg,"Atacante do Flamengo, Matheus Gonçalves no Atl...",https://ge.globo.com/futebol/times/atletico-mg...,"O atacante do Flamengo, Matheus Gonçalves, est..."
14,botafogo,Legado do capitão: Carli se arrepia com postur...,https://ge.globo.com/futebol/times/botafogo/no...,Carli entende bem qual é a responsabilidade de...
...,...,...,...,...
674,vitoria,Alerrandro fica no Vitória? Presidente abre o ...,https://www.cnnbrasil.com.br/esportes/futebol/...,NaN
675,vitoria,Vitória rescinde contrato com patrocinador más...,https://www.cnnbrasil.com.br/esportes/futebol/...,NaN
676,vitoria,Vitória anuncia renovação de contrato com o té...,https://www.cnnbrasil.com.br/esportes/futebol/...,NaN
677,vitoria,"Gol de Alerrandro, do Vitória, é eleito o mais...",https://www.cnnbrasil.com.br/esportes/futebol/...,NaN
